In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import split,col,filter,isnan,when
from pyspark.sql.types import StringType,IntegerType
import pandas as pd
import matplotlib.pyplot as plt
from petastorm.unischema import Unischema,UnischemaField,dict_to_spark_row
from petastorm.etl.dataset_metadata import materialize_dataset
from petastorm.codecs import ScalarCodec,NdarrayCodec
from petastorm.transform import TransformSpec
import numpy as np
import psutil
import os
import sys
sys.path.append(os.path.join(os.getcwd(), ".."))
from operations.dataset_manager import get_dataset

## Get your dataset 

In [ ]:
name = "mit-bih-noise-stress-test-database-1.0.0"
dataset = get_dataset(name,fs=500,time_window = 10)

## Create data parquet file (the parquet scheme will be given)

### Checking your cpu core and usage

In [ ]:
##Check number of cpu core and memory
print(os.cpu_count())
# Getting % usage of virtual_memory ( 3rd field)
print('RAM memory % used:', psutil.virtual_memory()[2])
# Getting usage of virtual_memory in GB ( 4th field)
print('RAM Used (GB):', psutil.virtual_memory()[3]/1000000000)

### Ceating a Spark session

In [ ]:
spark = SparkSession.builder.appName('MyfirstSpark').config('spark.driver.memory', '2g').getOrCreate()

### Unischema use (This is the one we use! please provide your unischema)

In [ ]:
ECGSchemaPhysio = Unischema(
    "ECGSchema",
    [
        UnischemaField("noun_id", np.string_, (), ScalarCodec(StringType()), False),
        UnischemaField("signal", np.float64, (None,None,None), NdarrayCodec(), False),
        UnischemaField("fs", np.int_, (), ScalarCodec(IntegerType()), False),
        UnischemaField("sig_len", np.int_, (), ScalarCodec(IntegerType()), False),
        UnischemaField("sig_name", np.string_, (None,), NdarrayCodec(), False),
        UnischemaField("n_sig", np.int_, (), ScalarCodec(IntegerType()), False),
        UnischemaField("base_date", np.string_, (), ScalarCodec(StringType()), True),
        UnischemaField("base_time", np.string_, (), ScalarCodec(StringType()), True),
        UnischemaField("units", np.string_, (None,), NdarrayCodec(), False),
        UnischemaField("comments", np.string_, (None,), NdarrayCodec(), False),
        UnischemaField("nb_time_window", np.int_, (), ScalarCodec(IntegerType()), False),
    ],
)

### Create path for saving


In [ ]:
test_path = "/workspaces/ecg_evaluation/data/mit-bih-noise-stress-test-database-1.0.0"
save_path = "/workspaces/ecg_evaluation/data/mit-bih-noise-stress-test-database-1.0.0/ParquetFile"

if not os.path.isdir(save_path):
    os.mkdir(save_path)

### Conversion of one sample into Parquet file

#### Conversion will be done using Petastorm

In [ ]:
row_group_size_mb = 256
sc = spark.sparkContext
rows_count = 10
test_data = dataset[0]
df_f = spark.createDataFrame(test_data,ECGSchemaPhysio.as_spark_schema())
df_f.show()
# with materialize_dataset(spark,save_path,ECGSchemaPhysio,row_group_size_mb):

#     spark.createDataFrame(rows_rdd,ECGSchemaPhysio.as_spark_schema())\
#         .coalesce(10)\
#         .write\
#         .mode("overwrite")\
#         .parquet(save_path)